# CLIP: Localized Sea Level Rise

This python notebook downloads the necessary data for visualizing sea level rise (temporal extent 2002-01 -2019-12) from this URL (https://www.seanoe.org/data/00631/74354/) and converts it into a format that is usable for Datawrapper or other visualization libraries. It creates a mean of the local_sla_trend value over the temporal extent. 

It requires standard modules like os, and shutil. Modules like requests, pandas and xarray might have to be installed depending on local setup & environment.

## Imports

In [64]:
import os
import shutil
import requests
import pandas as pd
import xarray as xr

## Data Download

In [65]:
url = 'https://www.seanoe.org/data/00631/74354/data/98856.zip'

req = requests.get(url)
filename = 'sla_data.zip'

with open(filename,'wb') as output_file:
    output_file.write(req.content)

In [66]:
unzip_path = "nc_files"
isExist = os.path.exists(unzip_path)
if not isExist:
   os.makedirs(unzip_path)

In [67]:
shutil.unpack_archive('sla_data.zip', unzip_path)

## Data Processing

In [69]:
#iterate over nc files and extract datapoints per station
df_viz = pd.DataFrame()

for root, _, files in os.walk(unzip_path):
    for x in files:
        if x.endswith('.nc'):
          print(x)
          x = 'nc_files/' + x

          ds = xr.open_dataset(x)

          ds["year"] = ds["time"].dt.strftime("%Y")
          ds["month"] = ds["time"].dt.strftime("%m")

          df = ds.to_dataframe().reset_index()
          df = df[['lat', 'lon', 'year', 'month', 'local_sla_trend']].copy()

          df_viz = df_viz.append(df, ignore_index=True)

ESACCI-SEALEVEL-IND-MSLTR-MERGED-N_INDIAN_JA_155_03-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-NE_ATL_JA_222_03-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-GULFSTREAM_JA_252_05-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-WAFRICA_JA_122_02-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-NW_AMERICA_JA_104_07-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-MED_SEA_JA_222_03-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-CARIBBEAN_JA_178_01-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-SE_ASIA_JA_164_01-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-WAFRICA_JA_020_02-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-WAFRICA_JA_122_03-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-SE_ASIA_JA_088_01-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-SE_ASIA_JA_101_05-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-CARIBBEAN_JA_141_02-20221220-fv02.2.nc
ESACCI-SEALEVEL-IND-MSLTR-MERGED-CARIBBEAN_JA_193_01-20221220-fv02.2.nc
ESACCI-S

## Data Export for Visualization

In [70]:
# create dataframe for visualization

# round all values to 2 digits, group and mean by coordinates and round again
df_viz = df_viz.round(2)
df_viz = df_viz.groupby(['lat', 'lon'])['local_sla_trend'].mean().reset_index().round(2)

# filter European stations (change coordinate boundaries for other regions)
df_viz = df_viz.loc[(df_viz['lat'] > 30.0) & (df_viz['lat'] < 64.0)]
df_viz = df_viz.loc[(df_viz['lon'] > -30.0) & (df_viz['lon'] < 70.0)]

# create categories
# calculate the minimum and maximum values
min_value = df_viz['local_sla_trend'].min()
max_value = df_viz['local_sla_trend'].max()

# define the bin edges
bin_edges = pd.cut(df_viz['local_sla_trend'], bins=5, labels=False, retbins=True)[1]

# categorize values into 5 classes using the `cut` function
df_viz['class'] = pd.cut(df_viz['local_sla_trend'], bins=bin_edges, labels=False)

# add class labels
df_viz['class'] = 'class_' + (df_viz['class'] + 1).astype(str)

# save as csv
df_viz.to_csv('viz_sla.csv', index=False)

# show preview
print(df_viz.head())

        lat    lon  local_sla_trend    class
9954  30.42  18.79             0.92  class_3
9955  30.43  18.79             0.82  class_3
9956  30.43  18.80             0.51  class_3
9957  30.44  18.80             0.59  class_3
9958  30.45  18.80             1.93  class_3


## Data Visualization

In [74]:
import altair as alt

alt.Chart(df_viz).mark_circle().encode(
    x=alt.X('lon', sort=alt.EncodingSortField(field='lon', order='ascending'), title=None, scale=alt.Scale(domain=[-15, 40]), axis=None, bin=alt.Bin(maxbins=15)),
    y=alt.Y('lat', sort=alt.EncodingSortField(field='lat', order='descending'), title=None, scale=alt.Scale(domain=[30, 60]), axis=None, bin=alt.Bin(maxbins=15)),
    size=alt.Size('local_sla_trend:Q', legend=None),
    color=alt.Color('local_sla_trend:Q', legend=None),
    tooltip='local_sla_trend:Q'
).configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
).properties(
    width=600,
    height=500
)

alt.Chart(...)